In [5]:
import pandas as pd
import numpy as np
import re
import os
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
import faiss

In [ ]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   --- ------------------------------------ 1.3/14.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.9 MB 5.9 MB/s eta 0:00:03
   ------- -------------------------------- 2.9/14.9 MB 4.4 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.9 MB 4.0 MB/s eta 0:00:03
   ---------- ----------------------------- 3.9/14.9 MB 3.6 MB/s eta 0:00:04
   ----------- ---------------------------- 4.5/14.9 MB 3.3 MB/s eta 0:00:04
   ------------- -------------------------- 5.0/14.9 MB 3.3 MB/s eta 0:00:03
   --------------- ------------------------ 5.8/14.9 MB 3.3 MB/s eta 0:00:03
   ---------------- ----------------------- 6.3/14.9 MB 3.2 MB/s eta 0:00:03
   ------------------ --------------------- 6.8/14.9 MB 3.2 MB/s eta 0:00:03
   ------------------- -------------------- 7.1/14.9 MB 3.2 MB/s eta 0:00:03
   ----------

In [15]:
ls

 El volumen de la unidad C es OS
 El n�mero de serie del volumen es: 14C8-5E4E

 Directorio de c:\Users\karen\Documents\HumanidadesDigitales_git\C�digos

11/08/2025  00:08    <DIR>          .
09/08/2025  16:05    <DIR>          ..
09/08/2025  15:59    <DIR>          archivos_txt
09/08/2025  16:07        28.552.361 archivos_txt.zip
28/07/2025  00:18        30.437.819 columnas_procesadas.json
09/08/2025  15:58        21.306.801 Corpus_completo.xlsx
11/08/2025  05:41             5.983 InformationRetrieval.ipynb
28/07/2025  16:52           122.055 lda_visualization.html
28/07/2025  01:52        20.545.144 probs_bert.json
28/07/2025  01:52       197.568.767 topic_model.pkl
09/08/2025  16:07           211.212 topic_modeling.ipynb
28/07/2025  01:52           116.254 topics_bert.json
               9 archivos    298.866.396 bytes
               3 dirs  862.397.616.128 bytes libres


In [6]:
file_path = "Corpus_completo.xlsx"
corpus_completo = pd.read_excel(file_path)


In [7]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

In [26]:
corpus_completo

,Diario,Autor,Fecha,Título,Texto,Vínculo,ID,Fecha_formateada
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,1,2018-01-01
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,2,2018-01-01
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,3,2018-01-01
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...,4,2018-01-01
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...,5,2018-01-01
...,...,...,...,...,...,...,...,...
13337,Semana,Esteban Piedrahita,25 de diciembre de 2018,Capitalismo a la escandinava,Los países nórdicos coquetearon con el sociali...,https://web.archive.org/web/20181227221015/htt...,13338,2018-12-25
13338,Semana,Germán Manga,26 de diciembre de 2018,"Ahora, a derrotar a los Guachos del Amazonas",Solo una acción militar sostenida y de gran es...,https://web.archive.org/web/20181227220937/htt...,13339,2018-12-26
13339,Semana,Ariel Ávila,26 de diciembre de 2018,Porte de armas y Estado de derecho,El debate del porte de armas es más complejo d...,https://web.archive.org/web/20181227220928/htt...,13340,2018-12-26
13340,Semana,Lucas Pombo,27 de diciembre de 2018,Quién es quién en el Congreso,"En estos primeros meses de actividad, algunos ...",https://web.archive.org/web/20181227220919/htt...,13341,2018-12-27


In [8]:
# Generar embeddings de textos
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [ ]:
# elegir  3000 columnas
textos = corpus_completo['Texto'][:3000].astype(str).tolist()  
embeddings = modelo_embedding.encode(textos, convert_to_tensor=False).astype('float32')


0.081326604

In [28]:
print(textos[1000])

Nunca estuvo en Japón. Sobre este país se tenía un conocimiento limitado. El hombre fuerte de turno concedió en 1609 a los holandeses el privilegio de ser los únicos occidentales a los que se les permitía comerciar con el Imperio del Sol Naciente. Esta prerrogativa duró algo más de 200 años. Hacia 1853 todo cambió y esa nación se abrió al comercio con varios países que estaban listos para penetrar en un mundo poco conocido. Japón se convirtió en algo de moda. Sus costumbres y su visión del arte impactaron a una clase de artistas, entre ellos al gran pintor holandés Vincent van Gogh.

Para un lego en la materia como yo, las pinturas japonesas tienen unas características básicas, son simples y minimalistas. El pajarito lleno de colores en la rama de un árbol, un cerezo en flor, una escena en el campo, bellas mujeres ataviadas con coloridos kimonos. Expresiones que producen tranquilidad, la armonía entre la pintura y el artista. Fue este uno de los aspectos que a van Gogh le llegó más pro

In [47]:
# Crear índice FAISS
indice = faiss.IndexFlatL2(embeddings.shape[1])
indice.add(embeddings)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000024E7693FB40> >

In [105]:
# Buscar texto similar a una consulta
consulta = ["cien años de soledad"]
embedding_consulta = modelo_embedding.encode(consulta, convert_to_tensor=False).astype('float32')
distancias, indices = indice.search(embedding_consulta, k=100)

In [106]:
distancias

array([[5.649014 , 6.2767773, 6.4750037, 6.834414 , 6.9755573, 6.9843607,
        7.036621 , 7.120755 , 7.264022 , 7.329239 , 7.5389023, 7.6039734,
        7.6153464, 7.695842 , 7.7006636, 7.708638 , 7.7321477, 7.758495 ,
        7.7958994, 7.808132 , 7.840538 , 7.849592 , 7.8548684, 7.8602724,
        7.8962355, 7.93069  , 7.949158 , 7.95421  , 7.9566956, 7.961074 ,
        7.971554 , 7.973383 , 8.002287 , 8.007851 , 8.014166 , 8.036582 ,
        8.041397 , 8.041659 , 8.051445 , 8.094881 , 8.101324 , 8.110939 ,
        8.112045 , 8.127706 , 8.133202 , 8.167383 , 8.171066 , 8.172409 ,
        8.172779 , 8.174431 , 8.178423 , 8.18078  , 8.180967 , 8.199382 ,
        8.208199 , 8.230658 , 8.237173 , 8.245958 , 8.258532 , 8.267467 ,
        8.2738085, 8.275563 , 8.282621 , 8.2831955, 8.283205 , 8.288397 ,
        8.311848 , 8.31478  , 8.316139 , 8.3185005, 8.329556 , 8.331049 ,
        8.335808 , 8.343429 , 8.350565 , 8.357697 , 8.362621 , 8.372183 ,
        8.372759 , 8.378206 , 8.386494

In [107]:
indices[0]  # Índices de los textos más similares

array([ 207, 2155, 2418, 1478, 1506,  488, 2877, 2308, 2055, 2739, 1059,
       1832,  229,  186,  619, 2744,  755, 2138, 2806,  643, 1503, 1960,
       2726, 1109, 2568, 1121, 1049, 2435, 1303, 2492,  111,  138,   82,
       2196,  998, 2326,  182, 1950, 1508, 1252, 2828, 2639,   34, 2710,
       2190, 1746,  496, 2907, 2791,   49, 2204, 1986,  613, 2212,  343,
       1867, 2728, 2529, 1912, 1811,  703,  608, 1016,  958,   66, 2680,
        549, 1428, 1427, 1204,  922,  397,  329, 1881,  719, 1261, 1627,
        214,  475, 1880,  845, 1996, 1553,   40,    3, 2329,  239, 2849,
       2220, 1155,  573,  408,  269,  469, 2880, 1938, 2512, 1250,  831,
       1081], dtype=int64)

In [104]:
# Filtrar la fila

corpus_completo.iloc[306]

Diario                                                  El Espectador
Autor                                            Esteban Carlos Mejía
Fecha                                             26 de enero de 2018
Título                          Jáuregui & Robles: curiosidad y deseo
Texto               En lo más recóndito de nuestros corazoncitos, ...
Vínculo             https://web.archive.org/web/20180128012313/htt...
ID                                                                307
Fecha_formateada                                  2018-01-26 00:00:00
Name: 306, dtype: object

In [108]:
#Resultados
print("Textos más similares:")
for idx in indices[0]:
    print(f"Índice: {idx}")
    print("Texto:" + textos[idx])
    print("---------------------------------------")


Textos más similares:
Índice: 207
Texto:La soledad, en el mundo rico, es una epidemia devastadora. Es como fumarse quince cigarrillos diarios. Peor que la obesidad, eleva en una cuarta parte la posibilidad de morirse. Estar solo se convirtió en problema de salud pública, necesitado de políticas oficiales.

El informe internacional publicado ayer por El Espectador es espeluznante. En Inglaterra, cada semana diez mil personas llaman a una línea telefónica especializada en oír. ¨Quieren hablar, no tienen a nadie que los escuche¨. Las cifras en toda Europa son para temblar.

Gente que no tiene a nadie, que en caso de emergencia carecen de un amigo o conocido a quien pedir ayuda. O con quien conversar. Miles, millones de personas que pasan un año sin hablar con otro ser humano. Por eso se habla de ´la enfermedad del silencio´.

Es la bomba atómica espiritual. Cada persona es un islote en el cosmos. Desprovista de lazos cordiales, flamea igual que aerolito loco en la vastedad. La explosión d